> Data <br>
https://www.kaggle.com/code/suyashlakhani/fake-news-classifier-nlp-98/notebook

- 자연어처리에 CNN을 적용시켜보고싶었음

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import string
import nltk

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.snowball import FrenchStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
fake_data = pd.read_csv("/Users/jacob/Documents/CodeStates/Section_4/Project/eng_dataset/Fake.csv")
true_data = pd.read_csv("/Users/jacob/Documents/CodeStates/Section_4/Project/eng_dataset/True.csv")

In [3]:
#1. Data Preprocessing
#Creating labels for classification 
fake_data["Label"] = 0
true_data["Label"] = 1

In [4]:
data = pd.concat([fake_data,true_data], axis=0,ignore_index = True)
# data = pd.concat([fake_data.iloc[:2000,:],true_data.iloc[:2000,:]], axis=0,ignore_index = True)

In [5]:
data.head()

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [6]:
data.shape

(44898, 5)

In [7]:
data.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east', 'politicsNews', 'worldnews'], dtype=object)

In [8]:
data.isnull().sum()

title      0
text       0
subject    0
date       0
Label      0
dtype: int64

In [9]:
data.drop(["title","subject","date"], axis=1, inplace=True)

In [10]:
data.text[3]

'On Christmas day, Donald Trump announced that he would  be back to work  the following day, but he is golfing for the fourth day in a row. The former reality show star blasted former President Barack Obama for playing golf and now Trump is on track to outpace the number of golf games his predecessor played.Updated my tracker of Trump s appearances at Trump properties.71 rounds of golf including today s. At this pace, he ll pass Obama s first-term total by July 24 next year. https://t.co/Fg7VacxRtJ pic.twitter.com/5gEMcjQTbH  Philip Bump (@pbump) December 29, 2017 That makes what a Washington Post reporter discovered on Trump s website really weird, but everything about this administration is bizarre AF. The coding contained a reference to Obama and golf:  Unlike Obama, we are working to fix the problem   and not on the golf course.  However, the coding wasn t done correctly.The website of Donald Trump, who has spent several days in a row at the golf course, is coded to serve up the fo

In [11]:
def remove_punct(_str):
#     _str = re.sub('['+string.punctuation+']', ' ', _str)
#     _str = re.sub('[\r]', ' ', _str)
#     _str = re.sub('[\n]', ' ', _str)
#     _str = re.sub("[^a-zA-Z]",' ',_str)
#     _str = _str.lower()
    
    _str = re.sub("[^a-zA-Z]",' ',_str)
    _str = _str.lower()
    _str = _str.split()
    
    return _str

In [12]:
data['text'] = data['text'].apply(remove_punct)


In [13]:
eng_stopwords = set(stopwords.words('english'))
filtre_stopfr = lambda text: [token for token in text if token.lower() not in eng_stopwords]


In [14]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['Label'], test_size=0.30, random_state=42)


In [15]:
tokenizer = Tokenizer(num_words=60000, lower=None)
tokenizer.fit_on_texts(data['text'])

seq_train = tokenizer.texts_to_sequences(X_train)
seq_test = tokenizer.texts_to_sequences(X_test)


In [16]:
print("Nb of texts: " , tokenizer.document_count)
print("Word/token counts: " , len(tokenizer.word_counts))
nb_token = len(tokenizer.word_index)

Nb of texts:  44898
Word/token counts:  115491


In [17]:
ds_train = pad_sequences(seq_train)
train_seq_len = ds_train.shape[1]
ds_train.shape

(31428, 8244)

In [18]:
ds_test = pad_sequences(seq_test, maxlen=train_seq_len)
ds_test.shape

(13470, 8244)

In [19]:
train_seq_len, nb_token

(8244, 115491)

In [20]:
ds_train

array([[    0,     0,     0, ...,   761,   295,  6150],
       [    0,     0,     0, ...,   239,     2,   404],
       [    0,     0,     0, ..., 14933,   303, 22007],
       ...,
       [    0,     0,     0, ...,     4,  1224,  2761],
       [    0,     0,     0, ...,   554,   490,  3265],
       [    0,     0,     0, ...,   157,  1033,   100]], dtype=int32)

In [21]:
ds_train.shape

(31428, 8244)

In [22]:
model = tf.keras.Sequential()

model.add(Input(shape=(train_seq_len,)))

model.add(Embedding(nb_token + 1, 20))

model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(3))

model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))

model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))


Metal device set to: Apple M1


2022-10-04 15:50:29.707055: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-04 15:50:29.707177: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8244, 20)          2309840   
                                                                 
 conv1d (Conv1D)             (None, 8242, 32)          1952      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2747, 32)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 2745, 64)          6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 915, 64)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 913, 128)          2

In [25]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)
model.fit(x=ds_train, 
            y=y_train, 
            validation_data=(ds_test, y_test), 
            epochs=5
            ,callbacks=[early_stop]
            )

Epoch 1/5


2022-10-04 15:50:29.930649: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-04 15:50:30.819880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


983/983 [==============================] - ETA: 0s - loss: 0.0675 - accuracy: 0.9636

2022-10-04 16:43:33.539863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


983/983 [==============================] - 3191s 3s/step - loss: 0.0675 - accuracy: 0.9636 - val_loss: 0.0028 - val_accuracy: 0.9992
Epoch 2/5
983/983 [==============================] - 1008s 1s/step - loss: 0.0020 - accuracy: 0.9995 - val_loss: 0.0020 - val_accuracy: 0.9994
Epoch 3/5
983/983 [==============================] - 420s 427ms/step - loss: 0.1142 - accuracy: 0.9181 - val_loss: 0.6923 - val_accuracy: 0.5264
Epoch 4/5
941/983 [===========================>..] - ETA: 3s - loss: 0.6924 - accuracy: 0.5203

KeyboardInterrupt: 